# Data Importing Using data_loader.ipynb

In [1]:
#from dataloaders import Dataloader as DL

import os
from pathlib import Path
import h5py
import pandas as pd
import numpy as np
import arpespythontools as arp
import matplotlib.pyplot as plt
from ipywidgets import widgets, Layout, interactive_output
import astropy 
from mpl_toolkits.mplot3d import Axes3D
import pickle
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from datetime import datetime
import csv

In [2]:
# Run the notebook script
%run data_loader.ipynb

# Access the cached data directly by variable names
map_data2 = map_data2
phi = phi
energy = energy
theta = theta
theta_v = theta_v


Loaded data from cache: 20230317_00029_cache.pkl


# ANGLE TO WAVEVECTOR

In [3]:
def angle_to_wavevector_binding(E_pho, work_f, theta, phi):
    #CONVERTS PIXELS TO THETA
    global theta_v
    
    #print(theta_v)
    E_k = E_pho - work_f
    k = 0.512*np.sqrt(E_k)
    theta_rad = np.radians(theta_v)
    phi_rad = np.radians(phi)
    
    #K_x and K_Y 
    k_x = k*np.sin(theta_rad)
    k_y = k*np.sin(phi_rad)
    
    #k_x = np.degrees(k_x)
    #k_y = np.degrees(k_y)
    return k_x, k_y
        
    
global k_x
global k_y

k_x, k_y = angle_to_wavevector_binding(85, 4, theta_v, phi)

# Variables

In [4]:
# Set global parameters
global emin1, emax2, vmax1, k_x1, k_x2, vmax2, pmin, pmax, vmax, k_y1, k_y2
emin1, emax2, vmax1 = -0.44, -0.41, 16226
k_x1, k_x2, vmax2 = -1, 360, 50000
k_y1, k_y2 = 0, 121
pmin, pmax, vmax = -1, 2, 35480
current_date = datetime.now().strftime("%Y-%m-%d")

# Scan Name 

In [5]:
file_path = data_file_path

# Function to extract dataset name under '2D_Data' group
def get_dataset_name(file_path):
    with h5py.File(file_path, "r") as h5_file:
        if "2D_Data" in h5_file:
            dataset_names = list(h5_file["2D_Data"].keys())  # Extract dataset names
            return dataset_names[0]  # Assuming you want the first dataset name
        else:
            return "Group '2D_Data' not found in the file."

# Get the dataset name
dataset_name = str(get_dataset_name(file_path))

# Output the dataset name
print(f"The dataset name is: {dataset_name}")

The dataset name is: Swept_Spectra46


## View Full Data To See Where to Slice

In [6]:
def plt_plot(Vmax = 50000, k_x_min = -1, k_x_max = 2):
    
    surface1 = arp.plane_slice(map_data2.transpose([1, 0,2]), k_x, k_x_min, k_x_max)
    plt.figure(figsize = (8, 6))
    plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', extent = [k_y[0], k_y[-1], energy[0], energy[-1]], cmap = 'gray')
    plt.xlabel("Wavevector (k_y)")
    plt.ylabel("Energy")
    plt.colorbar()
    

widgets.interact(plt_plot, Vmin = 0, Vmax = (0, 1000000, 0.01), k_x_min = (k_x[0], k_x[-1], 0.01), 
                 k_x_max = (k_x[0], k_x[-1], 0.01))

interactive(children=(FloatSlider(value=50000.0, description='Vmax', max=1000000.0, step=0.01), FloatSlider(va…

<function __main__.plt_plot(Vmax=50000, k_x_min=-1, k_x_max=2)>

In [7]:
def plt_plot(Vmax = 35480, k_y_min = -1, k_y_max = 2):
    
    surface1 = arp.plane_slice(map_data2.transpose([2, 0, 1]), k_y, k_y_min, k_y_max)
    plt.figure(figsize = (8, 6))
    plt.imshow(surface1, aspect = "auto", vmax = Vmax, origin = 'lower', extent = [k_x[0], k_x[-1], energy[0], energy[-1]], cmap = 'gray')
    plt.xlabel("Wavevector (k_x)")
    plt.ylabel("Energy")

    #plt.xlim(-2, 2)
    #plt.ylim(-1.3, -0.25)
    plt.colorbar()
    
    

widgets.interact(plt_plot, Vmin = 0, Vmax = (0, 1000000, 0.01), k_y_min = (k_y[0], k_y[-1], 0.01), 
                 k_y_max = (k_y[0], k_y[-1], 0.01))

interactive(children=(FloatSlider(value=35480.0, description='Vmax', max=1000000.0, step=0.01), FloatSlider(va…

<function __main__.plt_plot(Vmax=35480, k_y_min=-1, k_y_max=2)>

# Waterfall plot without color (Slicing K_Y)

In [8]:
# Function to add bounds to the CSV file and manage folder structure
def add_bounds_to_csv(k_y1, k_y2, Ev_Lowerbound, Ev_Upperbound, current_date):
    global dataset_name
    # Ensure main folder exists
    main_folder = "ARPES Waterfall Plots"
    os.makedirs(main_folder, exist_ok=True)
    
    # Create subfolder for the dataset
    subfolder = os.path.join(main_folder, dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Create or append to the CSV file in the subfolder
    file_name = os.path.join(subfolder, f"{dataset_name}_k_y_bounds.csv")
    bounds_df = pd.DataFrame({
        "Energy_Lowerbound": [Ev_Lowerbound],
        "Energy_Upperbound": [Ev_Upperbound],
        "k_y_Lowerbound": [k_y1],
        "k_y_Upperbound": [k_y2],
        "Date": [current_date]
    })
    try:
        bounds_df.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
        print(f"Bounds added to {file_name}")
    except PermissionError:
        print(f"Permission denied: Unable to write to {file_name}. Please check file permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to generate the plot and save data based on inputs
def plot1d(k_y1, k_y2, offset, interval, Ev_Lowerbound, Ev_Upperbound):
    global dataset_name

    # Create a surface slice for the given range
    surface = arp.plane_slice(map_data2.transpose([1, 0, 2]), k_x, k_x1, k_x2)
    
    # Initialize figure
    fig, ax = plt.subplots(figsize=(8, 6))  # Create figure and axis explicitly

    # Loop over slices to create a waterfall effect
    for i, ky in enumerate(np.arange(k_y1, k_y2, interval)):
        energy_line = arp.line_profile(surface, k_y, ky, ky + 0.01)
        energy_line = energy_line / np.max(energy_line)  # Normalize intensity for plotting
        ax.plot(energy, energy_line + i * offset, color='green', label=f"k_y={ky:.2f}")

    ax.set_xlim(Ev_Lowerbound, Ev_Upperbound)
    ax.set_ylabel("Intensity + Offset")
    ax.set_xlabel("Energy (eV)")
    ax.set_title(f"1D Waterfall Plot - {dataset_name}")
    plt.close()
    return fig  # Return the figure object

def plot1d_save_data(k_y1, k_y2, offset, interval, Ev_Lowerbound, Ev_Upperbound):
    global dataset_name

    
    surface = arp.plane_slice(map_data2.transpose([1, 0, 2]), k_x, k_x1, k_x2)
    
    data_dict = {}  

   
    for i, ky in enumerate(np.arange(k_y1, k_y2, interval)):
        energy_line = arp.line_profile(surface, k_y, ky, ky + 0.01)
        energy_line = energy_line / np.max(energy_line)  # Normalize intensity for plotting
        
        # Store energy and intensity values in a dictionary
        data_dict[ky] = list(zip(energy, energy_line + i * offset))

    # Determine the maximum number of energy values in any k_y group
    max_len = max(len(v) for v in data_dict.values())

    # Prepare header row
    header = []
    for ky in sorted(data_dict.keys()):
        header.append(f"Energy k_y({ky:.2f})")
        header.append(f"Intensity k_y({ky:.2f})")

    # Prepare data rows
    formatted_data = []
    for row_idx in range(max_len):
        row = []
        for ky in sorted(data_dict.keys()):
            energy_intensity_list = data_dict[ky]
            if row_idx < len(energy_intensity_list):
                row.extend(energy_intensity_list[row_idx])
            else:
                row.extend(["", ""])  # Fill empty spaces if k_y has fewer values
        formatted_data.append(row)

    # Create subfolder for saving the CSV file
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Generate a unique file name using parameter values
    unique_id = f"k_y1_{k_y1:.2f}_k_y2_{k_y2:.2f}_E_{Ev_Lowerbound:.2f}_{Ev_Upperbound:.2f}"
    csv_filename = os.path.join(subfolder, f"{dataset_name}_{unique_id}.csv")

    # Save data to CSV
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write header
        writer.writerows(formatted_data)  # Write the transposed data

    print(f"CSV data saved successfully to {csv_filename}")

    
def save_plot(figure):
    global dataset_name
    # Construct subfolder
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Determine the current row number in the CSV
    csv_file = os.path.join(subfolder, f"{dataset_name}_bounds.csv")
    if os.path.exists(csv_file):
        try:
            existing_df = pd.read_csv(csv_file)
            row_number = len(existing_df)  # Current row number
        except Exception as e:
            print(f"Error reading CSV: {e}")
            row_number = 1
    else:
        row_number = 1

    # Generate a unique file name with row number
    unique_id = f"k_y1_{k_y1_slider.value:.2f}_k_y2_{k_y2_slider.value:.2f}_E_{Ev_Lowerbound_slider.value:.2f}_{Ev_Upperbound_slider.value:.2f}"
    file_name = os.path.join(subfolder, f"{dataset_name}_{unique_id}.png")

    # Save the figure
    figure.savefig(file_name, dpi=1200)
    print(f"Plot saved as {file_name}")


save_button = widgets.Button(description="Save Plot as PNG")
save_button.on_click(lambda _: save_plot(plot1d(
    k_y1_slider.value, k_y2_slider.value,  # Pass slider values
    offset_slider.value, interval_slider.value, 
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
)))
# Button to save CSV of Graph
save_csv_button = widgets.Button(description="Save CSV of Plot")

# Define the action for the "Save CSV Only" button
def save_csv_action(_):
    plot1d_save_data(
        k_y1_slider.value, k_y2_slider.value, offset_slider.value, interval_slider.value, 
        Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
    )

# Attach the action to the button
save_csv_button.on_click(save_csv_action)


# Button for saving the full plot data
save_plot_button = widgets.Button(description="Plot Data as CSV")
save_plot_button.on_click(lambda _: plot1d_save_data(
    k_y1_slider.value, k_y2_slider.value, offset_slider.value, interval_slider.value, 
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
))

# Button to add bounds to the CSV file
add_bounds_button = widgets.Button(description="Add Bounds to CSV")
add_bounds_button.on_click(lambda _: add_bounds_to_csv(
    k_y1_slider.value, k_y2_slider.value, Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value, current_date))

# Display the buttons
display(widgets.HBox([add_bounds_button, save_button, save_plot_button]))

# Create sliders for each parameter and bind them to widgets.interact
k_y1_slider = widgets.FloatSlider(value=-1.19, min=-1.19, max=1.19, step=0.01, description='k_y1')
k_y2_slider = widgets.FloatSlider(value=1.19, min=-1.19, max=1.19, step=0.01, description='k_y2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=-1, min=energy[0], max=energy[-1], step=0.01, description='Lower Energy')
Ev_Upperbound_slider = widgets.FloatSlider(value=-0.4, min=energy[0], max=energy[-1], step=0.01, description='Upper Energy')
offset_slider = widgets.FloatSlider(value=0.2, min=0.01, max=1.0, step=0.05, description='Offset')
interval_slider = widgets.FloatSlider(value=0.05, min=0.001, max=0.5, step=0.01, description='Interval')

widgets.interact(
    plot1d,
    k_y1=k_y1_slider,
    k_y2=k_y2_slider,
    offset=offset_slider,
    interval=interval_slider,
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider
)

interactive(children=(FloatSlider(value=-1.19, description='k_y1', max=1.19, min=-1.19, step=0.01), FloatSlide…

<function __main__.plot1d(k_y1, k_y2, offset, interval, Ev_Lowerbound, Ev_Upperbound)>

Bounds added to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_y_bounds.csv
Plot saved as ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_y1_-1.19_k_y2_1.19_E_-1.00_-0.40.png
CSV data saved successfully to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_y1_-1.19_k_y2_1.19_E_-1.00_-0.40.csv


# Waterfall plot without color (Slicing K_X)

In [9]:
def add_bounds_to_csv(k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, current_date):
    global dataset_name
    # Ensure main folder exists
    main_folder = "ARPES Waterfall Plots"
    os.makedirs(main_folder, exist_ok=True)

    # Create subfolder for the dataset
    subfolder = os.path.join(main_folder, dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Create or append to the CSV file in the subfolder
    file_name = os.path.join(subfolder, f"{dataset_name}_k_x_bounds.csv")
    bounds_df = pd.DataFrame({
        "Energy_Lowerbound": [Ev_Lowerbound],
        "Energy_Upperbound": [Ev_Upperbound],
        "k_x_Lowerbound": [k_x1],
        "k_x_Upperbound": [k_x2],
        "Date": [current_date]
    })
    try:
        bounds_df.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
        print(f"Bounds added to {file_name}")
    except PermissionError:
        print(f"Permission denied: Unable to write to {file_name}. Please check file permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")

def plot1d(k_x1, k_x2, offset, interval, Ev_Lowerbound, Ev_Upperbound):
    global dataset_name

    # Create a surface slice for the given range (using k_x slicing correctly)
    surface = arp.plane_slice(map_data2.transpose([2, 0, 1]), k_y, k_y1, k_y2) 
    
    # Initialize figure
    fig, ax = plt.subplots(figsize=(8, 6))  

    # Loop over slices to create a waterfall effect
    for i, kx in enumerate(np.arange(k_x1, k_x2, interval)):
        energy_line = arp.line_profile(surface, k_x, kx, kx + 0.01)
        energy_line = energy_line / np.max(energy_line)
        ax.plot(energy, energy_line + i * offset, color='green', label=f"k_x={kx:.2f}")

    ax.set_xlim(Ev_Lowerbound, Ev_Upperbound)
    ax.set_ylabel("Intensity + Offset")
    ax.set_xlabel("Energy (eV)")
    ax.set_title(f"1D Waterfall Plot - {dataset_name} (Slicing k_x)")
    plt.close()
    return fig  # Return the figure object

def plot1d_save_data(k_x1, k_x2, offset, interval, Ev_Lowerbound, Ev_Upperbound):
    global dataset_name

    
    surface = arp.plane_slice(map_data2.transpose([2, 0, 1]), k_y, k_y1, k_y2) 
    
    data_dict = {}  

   
    for i, kx in enumerate(np.arange(k_x1, k_x2, interval)):
        energy_line = arp.line_profile(surface, k_x, kx, kx + 0.01)
        energy_line = energy_line / np.max(energy_line)
        
        # Store energy and intensity values in a dictionary
        data_dict[kx] = list(zip(energy, energy_line + i * offset))

    # Determine the maximum number of energy values in any k_y group
    max_len = max(len(v) for v in data_dict.values())

    # Prepare header row
    header = []
    for kx in sorted(data_dict.keys()):
        header.append(f"Energy k_x({kx:.2f})")
        header.append(f"Intensity k_x({kx:.2f})")

    # Prepare data rows
    formatted_data = []
    for row_idx in range(max_len):
        row = []
        for kx in sorted(data_dict.keys()):
            energy_intensity_list = data_dict[kx]
            if row_idx < len(energy_intensity_list):
                row.extend(energy_intensity_list[row_idx])
            else:
                row.extend(["", ""])  # Fill empty spaces if k_y has fewer values
        formatted_data.append(row)

    # Create subfolder for saving the CSV file
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Generate a unique file name using parameter values
    unique_id = f"k_x1_{k_x1:.2f}_k_x2_{k_x2:.2f}_E_{Ev_Lowerbound:.2f}_{Ev_Upperbound:.2f}"
    csv_filename = os.path.join(subfolder, f"{dataset_name}_{unique_id}.csv")

    # Save data to CSV
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write header
        writer.writerows(formatted_data)  # Write the transposed data

    print(f"CSV data saved successfully to {csv_filename}")

    
def save_plot(figure):
    global dataset_name
    # Construct subfolder
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Determine the current row number in the CSV
    csv_file = os.path.join(subfolder, f"{dataset_name}_k_x_bounds.csv")
    if os.path.exists(csv_file):
        try:
            existing_df = pd.read_csv(csv_file)
            row_number = len(existing_df)  # Current row number
        except Exception as e:
            print(f"Error reading CSV: {e}")
            row_number = 1
    else:
        row_number = 1

    # Generate a unique file name with row number
    unique_id = f"k_x1_{k_x1_slider.value:.2f}_k_x2_{k_x2_slider.value:.2f}_E_{Ev_Lowerbound_slider.value:.2f}_{Ev_Upperbound_slider.value:.2f}"
    file_name = os.path.join(subfolder, f"{dataset_name}_{unique_id}.png")

    # Save the figure
    figure.savefig(file_name, dpi=1200)
    print(f"Plot saved as {file_name}")


save_button = widgets.Button(description="Save Plot as PNG")
save_button.on_click(lambda _: save_plot(plot1d(
    k_x1_slider.value, k_x2_slider.value,  # Pass slider values
    offset_slider.value, interval_slider.value, 
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
)))
# Button to save CSV of Graph
save_csv_button = widgets.Button(description="Save CSV of Plot")

# Define the action for the "Save CSV Only" button
def save_csv_action(_):
    plot1d_save_data(
        k_x1_slider.value, k_x2_slider.value, offset_slider.value, interval_slider.value, 
        Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
    )

# Attach the action to the button
save_csv_button.on_click(save_csv_action)


# Button for saving the full plot data
save_plot_button = widgets.Button(description="Plot Data as CSV")
save_plot_button.on_click(lambda _: plot1d_save_data(
    k_x1_slider.value, k_x2_slider.value, offset_slider.value, interval_slider.value, 
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
))

# Button to add bounds to the CSV file
add_bounds_button = widgets.Button(description="Add Bounds to CSV")
add_bounds_button.on_click(lambda _: add_bounds_to_csv(
    k_x1_slider.value, k_x2_slider.value, Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value, current_date))

# Display the buttons
display(widgets.HBox([add_bounds_button, save_button, save_plot_button]))    
    
# Create sliders for each parameter and bind them to widgets.interact
k_x1_slider = widgets.FloatSlider(value=-2, min=-4.49, max=4.49, step=0.01, description='k_x1')
k_x2_slider = widgets.FloatSlider(value=.5, min=-4.49, max=4.49, step=0.01, description='k_x2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=-1, min=energy[0], max=energy[-1], step=0.01, description='Lower Energy')
Ev_Upperbound_slider = widgets.FloatSlider(value=-0.4, min=energy[0], max=energy[-1], step=0.01, description='Upper Energy')
offset_slider = widgets.FloatSlider(value=0.2, min=0.01, max=5.0, step=0.001, description='Offset')
interval_slider = widgets.FloatSlider(value=0.05, min=0.001, max=0.5, step=0.01, description='Interval')

widgets.interact(
    plot1d,
    k_x1=k_x1_slider,
    k_x2=k_x2_slider,
    offset=offset_slider,
    interval=interval_slider,
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider
)

interactive(children=(FloatSlider(value=-2.0, description='k_x1', max=4.49, min=-4.49, step=0.01), FloatSlider…

<function __main__.plot1d(k_x1, k_x2, offset, interval, Ev_Lowerbound, Ev_Upperbound)>

Bounds added to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_x_bounds.csv
Plot saved as ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_x1_-2.00_k_x2_0.50_E_-1.00_-0.40.png
CSV data saved successfully to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_x1_-2.00_k_x2_0.50_E_-1.00_-0.40.csv


# Waterfall Graph With Intensity Scale (Slicing K_Y)

In [10]:

# Function to add bounds to the CSV file and manage folder structure
def add_bounds_to_csv(k_y1, k_y2, Ev_Lowerbound, Ev_Upperbound, current_date):
    global dataset_name
    # Ensure main folder exists
    main_folder = "ARPES Waterfall Plots"
    os.makedirs(main_folder, exist_ok=True)

    # Create subfolder for the dataset
    subfolder = os.path.join(main_folder, dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Create or append to the CSV file in the subfolder
    file_name = os.path.join(subfolder, f"{dataset_name}_k_y_bounds.csv")
    bounds_df = pd.DataFrame({
        "Energy_Lowerbound": [Ev_Lowerbound],
        "Energy_Upperbound": [Ev_Upperbound],
        "k_y_Lowerbound": [k_y1],
        "k_y_Upperbound": [k_y2],
        "Date": [current_date]
    })
    try:
        bounds_df.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
        print(f"Bounds added to {file_name}")
    except PermissionError:
        print(f"Permission denied: Unable to write to {file_name}. Please check file permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to generate the plot and save data based on inputs
def plot1d(k_y1=-1.19, k_y2=1.19, offset=60000, interval=0.25, Ev_Lowerbound=-1, Ev_Upperbound=-0.4):
    global dataset_name, fig, ax # Make the figure and axis global to access in save_plot

    # Simulate surface extraction and profile generation
    surface = arp.plane_slice(map_data2.transpose([1, 0, 2]), k_x, k_x1, k_x2)
    
    # Initialize figure and color map
    fig, ax = plt.subplots(figsize=(8, 6))
    cmap = cm.gray  # Choose a color map for intensity
    # Loop over slices to create the waterfall effect with offset
    for i, ky in enumerate(np.arange(k_y1, k_y2, interval)):
        energy_line = arp.line_profile(surface, k_y, ky, ky + 0.01)

        # Normalize intensity for color mapping based on intensity along the x-axis
        norm = mcolors.Normalize(vmin=np.min(energy_line), vmax= 1.15 * np.max(energy_line))
        
        # Plot line with colors varying by intensity at each x-point
        for j in range(len(energy) - 1):
            ax.plot(
                energy[j:j+2], 
                energy_line[j:j+2] + i * offset, 
                color=cmap(norm(energy_line[j])),
            )

    ax.set_xlim(Ev_Lowerbound, Ev_Upperbound)
    ax.set_ylabel("Intensity + Offset")
    ax.set_xlabel("Energy (eV)")

    # Add color bar to show the intensity scale
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=mcolors.Normalize(vmin=np.min(energy_line), vmax= 1.15*np.max(energy_line)))
    sm.set_array([])
    fig.colorbar(sm, ax=ax, label="Intensity")

    # Display the plot
    plt.show()

# Function to save the plot as a PNG file
def save_plot(_):
    global dataset_name, fig  # Ensure the figure is accessed from the global scope

    # Construct subfolder path
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Determine the current row number in the CSV
    csv_file = os.path.join(subfolder, f"{dataset_name}_bounds.csv")
    if os.path.exists(csv_file):
        try:
            existing_df = pd.read_csv(csv_file)
            row_number = len(existing_df)  # Current row number
        except Exception as e:
            print(f"Error reading CSV: {e}")
            row_number = 1
    else:
        row_number = 1

    # Generate a unique file name with row number
    unique_id = f"k_y1_{k_y1_slider.value:.2f}_k_y2_{k_y2_slider.value:.2f}_E_{Ev_Lowerbound_slider.value:.2f}_{Ev_Upperbound_slider.value:.2f}"
    file_name = os.path.join(subfolder, f"{dataset_name}_{unique_id}.png")

    # Save the figure
    if fig:
        fig.savefig(file_name, dpi=1200)
        print(f"Plot saved as {file_name}")
    else:
        print("No figure found to save!")

def plot1d_save_data(k_y1, k_y2, offset, interval, Ev_Lowerbound, Ev_Upperbound):
    global dataset_name

    
    surface = arp.plane_slice(map_data2.transpose([1, 0, 2]), k_x, k_x1, k_x2)
    
    data_dict = {}  

   
    for i, ky in enumerate(np.arange(k_y1, k_y2, interval)):
        energy_line = arp.line_profile(surface, k_y, ky, ky + 0.01)
        energy_line = energy_line / np.max(energy_line)  # Normalize intensity for plotting
        
        # Store energy and intensity values in a dictionary
        data_dict[ky] = list(zip(energy, energy_line + i * offset))

    # Determine the maximum number of energy values in any k_y group
    max_len = max(len(v) for v in data_dict.values())

    # Prepare header row
    header = []
    for ky in sorted(data_dict.keys()):
        header.append(f"Energy k_y({ky:.2f})")
        header.append(f"Intensity k_y({ky:.2f})")

    # Prepare data rows
    formatted_data = []
    for row_idx in range(max_len):
        row = []
        for ky in sorted(data_dict.keys()):
            energy_intensity_list = data_dict[ky]
            if row_idx < len(energy_intensity_list):
                row.extend(energy_intensity_list[row_idx])
            else:
                row.extend(["", ""])  # Fill empty spaces if k_y has fewer values
        formatted_data.append(row)

    # Create subfolder for saving the CSV file
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Generate a unique file name using parameter values
    unique_id = f"k_y1_{k_y1:.2f}_k_y2_{k_y2:.2f}_E_{Ev_Lowerbound:.2f}_{Ev_Upperbound:.2f}"
    csv_filename = os.path.join(subfolder, f"{dataset_name}_{unique_id}.csv")

    # Save data to CSV
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write header
        writer.writerows(formatted_data)  # Write the transposed data

    print(f"CSV data saved successfully to {csv_filename}")
# Create the save button globally
save_button = widgets.Button(description="Save Plot as PNG")
save_button.on_click(save_plot)

# Attach the action to the button
save_csv_button.on_click(save_csv_action)


# Button for saving the full plot data
save_plot_button = widgets.Button(description="Plot Data as CSV")
save_plot_button.on_click(lambda _: plot1d_save_data(
    k_y1_slider.value, k_y2_slider.value, offset_slider.value, interval_slider.value, 
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
))

# Button to add bounds to the CSV file
add_bounds_button = widgets.Button(description="Add Bounds to CSV")
add_bounds_button.on_click(lambda _: add_bounds_to_csv(
    k_y1_slider.value, k_y2_slider.value, Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value, current_date))

# Display the buttons
display(widgets.HBox([add_bounds_button, save_button, save_plot_button]))

# Create sliders for each parameter and bind them to widgets.interact
k_y1_slider = widgets.FloatSlider(value=-1.19, min=-2, max=2, step=0.01, description='k_y1')
k_y2_slider = widgets.FloatSlider(value=1.19, min=-2, max=2, step=0.01, description='k_y2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=-1, min=energy[0], max=energy[-1], step=0.01, description='Lower Energy')
Ev_Upperbound_slider = widgets.FloatSlider(value=-0.4, min=energy[0], max=energy[-1], step=0.01, description='Upper Energy')

widgets.interact(
    plot1d,
    k_y1=k_y1_slider,
    k_y2=k_y2_slider,
    offset=widgets.FloatSlider(value=60000, min=0.01, max=100000, step=0.05, description='Offset'),
    interval=widgets.FloatSlider(value=0.04, min=0.001, max=0.5, step=0.01, description='Interval'),
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider
)


interactive(children=(FloatSlider(value=-1.19, description='k_y1', max=2.0, min=-2.0, step=0.01), FloatSlider(…

<function __main__.plot1d(k_y1=-1.19, k_y2=1.19, offset=60000, interval=0.25, Ev_Lowerbound=-1, Ev_Upperbound=-0.4)>

Bounds added to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_y_bounds.csv
Plot saved as ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_y1_-1.19_k_y2_1.19_E_-1.00_-0.40.png
CSV data saved successfully to ARPES Waterfall Plots\Swept_Spectra46\Swept_Spectra46_k_y1_-1.19_k_y2_1.19_E_-1.00_-0.40.csv


# Waterfall Graph With Intensity Scale (Slicing K_X)

In [11]:

def add_bounds_to_csv(k_x1, k_x2, Ev_Lowerbound, Ev_Upperbound, current_date):
    global dataset_name
    # Ensure main folder exists
    main_folder = "ARPES Waterfall Plots"
    os.makedirs(main_folder, exist_ok=True)

    # Create subfolder for the dataset
    subfolder = os.path.join(main_folder, dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Create or append to the CSV file in the subfolder
    file_name = os.path.join(subfolder, f"{dataset_name}_k_x_bounds.csv")
    bounds_df = pd.DataFrame({
        "Energy_Lowerbound": [Ev_Lowerbound],
        "Energy_Upperbound": [Ev_Upperbound],
        "k_x_Lowerbound": [k_x1],
        "k_x_Upperbound": [k_x2],
        "Date": [current_date]
    })
    try:
        bounds_df.to_csv(file_name, mode='a', header=not os.path.exists(file_name), index=False)
        print(f"Bounds added to {file_name}")
    except PermissionError:
        print(f"Permission denied: Unable to write to {file_name}. Please check file permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Function to generate the plot and save data based on inputs
def plot1d(k_x1=-1.19, k_x2=1.19, offset=60000, interval=0.25, Ev_Lowerbound=-1, Ev_Upperbound=-0.4):
    global dataset_name, fig, ax # Make the figure and axis global to access in save_plot

    # Simulate surface extraction and profile generation
    surface = arp.plane_slice(map_data2.transpose([2, 0, 1]), k_y, k_y1, k_y2)
    
    # Initialize figure and color map
    fig, ax = plt.subplots(figsize=(8, 6))
    cmap = cm.gray  # Choose a color map for intensity
    # Loop over slices to create the waterfall effect with offset
    for i, kx in enumerate(np.arange(k_x1, k_x2, interval)):
        energy_line = arp.line_profile(surface, k_x, kx, kx + 0.01)

        # Normalize intensity for color mapping based on intensity along the x-axis
        norm = mcolors.Normalize(vmin=np.min(energy_line), vmax= 1.15 * np.max(energy_line))
        
        # Plot line with colors varying by intensity at each x-point
        for j in range(len(energy) - 1):
            ax.plot(
                energy[j:j+2], 
                energy_line[j:j+2] + i * offset, 
                color=cmap(norm(energy_line[j])),
            )

    ax.set_xlim(Ev_Lowerbound, Ev_Upperbound)
    ax.set_ylabel("Intensity + Offset")
    ax.set_xlabel("Energy (eV)")

    # Add color bar to show the intensity scale
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=mcolors.Normalize(vmin=np.min(energy_line), vmax=1.15 * np.max(energy_line)))
    sm.set_array([])
    fig.colorbar(sm, ax=ax, label="Intensity")

    # Display the plot
    plt.show()

# Function to save the plot as a PNG file
def save_plot(_):
    global dataset_name, fig  # Ensure the figure is accessed from the global scope

    # Construct subfolder path
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Determine the current row number in the CSV
    csv_file = os.path.join(subfolder, f"{dataset_name}_bounds.csv")
    if os.path.exists(csv_file):
        try:
            existing_df = pd.read_csv(csv_file)
            row_number = len(existing_df)  # Current row number
        except Exception as e:
            print(f"Error reading CSV: {e}")
            row_number = 1
    else:
        row_number = 1

    # Generate a unique file name with row number
    unique_id = f"k_x1_{k_x1_slider.value:.2f}_k_x2_{k_x2_slider.value:.2f}_E_{Ev_Lowerbound_slider.value:.2f}_{Ev_Upperbound_slider.value:.2f}"
    file_name = os.path.join(subfolder, f"{dataset_name}_{unique_id}.png")

    # Save the figure
    if fig:
        fig.savefig(file_name, dpi=1200)
        print(f"Plot saved as {file_name}")
    else:
        print("No figure found to save!")

def plot1d_save_data(k_x1, k_x2, offset, interval, Ev_Lowerbound, Ev_Upperbound):
    global dataset_name

    
    surface = arp.plane_slice(map_data2.transpose([2, 0, 1]), k_y, k_y1, k_y2) 
    
    data_dict = {}  

   
    for i, kx in enumerate(np.arange(k_x1, k_x2, interval)):
        energy_line = arp.line_profile(surface, k_x, kx, kx + 0.01)
        energy_line = energy_line / np.max(energy_line)
        
        # Store energy and intensity values in a dictionary
        data_dict[kx] = list(zip(energy, energy_line + i * offset))

    # Determine the maximum number of energy values in any k_y group
    max_len = max(len(v) for v in data_dict.values())

    # Prepare header row
    header = []
    for kx in sorted(data_dict.keys()):
        header.append(f"Energy k_x({kx:.2f})")
        header.append(f"Intensity k_x({kx:.2f})")

    # Prepare data rows
    formatted_data = []
    for row_idx in range(max_len):
        row = []
        for kx in sorted(data_dict.keys()):
            energy_intensity_list = data_dict[kx]
            if row_idx < len(energy_intensity_list):
                row.extend(energy_intensity_list[row_idx])
            else:
                row.extend(["", ""])  # Fill empty spaces if k_y has fewer values
        formatted_data.append(row)

    # Create subfolder for saving the CSV file
    subfolder = os.path.join("ARPES Waterfall Plots", dataset_name)
    os.makedirs(subfolder, exist_ok=True)

    # Generate a unique file name using parameter values
    unique_id = f"k_x1_{k_x1:.2f}_k_x2_{k_x2:.2f}_E_{Ev_Lowerbound:.2f}_{Ev_Upperbound:.2f}"
    csv_filename = os.path.join(subfolder, f"{dataset_name}_{unique_id}.csv")

    # Save data to CSV
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Write header
        writer.writerows(formatted_data)  # Write the transposed data

    print(f"CSV data saved successfully to {csv_filename}")

save_button = widgets.Button(description="Save Plot as PNG")
save_button.on_click(save_plot)

# Attach the action to the button
save_csv_button.on_click(save_csv_action)


# Button for saving the full plot data
save_plot_button = widgets.Button(description="Plot Data as CSV")
save_plot_button.on_click(lambda _: plot1d_save_data(
    k_x1_slider.value, k_x2_slider.value, offset_slider.value, interval_slider.value, 
    Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value
))

# Button to add bounds to the CSV file
add_bounds_button = widgets.Button(description="Add Bounds to CSV")
add_bounds_button.on_click(lambda _: add_bounds_to_csv(
    k_x1_slider.value, k_x2_slider.value, Ev_Lowerbound_slider.value, Ev_Upperbound_slider.value, current_date))

# Display the buttons
display(widgets.HBox([add_bounds_button, save_button, save_plot_button]))

# Create sliders for each parameter and bind them to widgets.interact
k_x1_slider = widgets.FloatSlider(value=-2, min=-4.50, max=4.49, step=0.01, description='k_x1')
k_x2_slider = widgets.FloatSlider(value=.5, min=-4.50, max=4.49, step=0.01, description='k_x2')
Ev_Lowerbound_slider = widgets.FloatSlider(value=-1, min=energy[0], max=energy[-1], step=0.01, description='Lower Energy')
Ev_Upperbound_slider = widgets.FloatSlider(value=-0.4, min=energy[0], max=energy[-1], step=0.01, description='Upper Energy')

widgets.interact(
    plot1d,
    k_x1=k_x1_slider,
    k_x2=k_x2_slider,
    offset=widgets.FloatSlider(value=60000, min=0.01, max=100000, step=0.05, description='Offset'),
    interval=widgets.FloatSlider(value=0.04, min=0.001, max=0.5, step=0.01, description='Interval'),
    Ev_Lowerbound=Ev_Lowerbound_slider,
    Ev_Upperbound=Ev_Upperbound_slider
)


interactive(children=(FloatSlider(value=-2.0, description='k_x1', max=4.49, min=-4.5, step=0.01), FloatSlider(…

<function __main__.plot1d(k_x1=-1.19, k_x2=1.19, offset=60000, interval=0.25, Ev_Lowerbound=-1, Ev_Upperbound=-0.4)>